# IMU

Acceleration data from the IMU.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, argmax
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

time = []
acc_x = []
acc_y = []
acc_z = []
with open("../telemetry.csv", 'r') as f_in:
    for line, text in enumerate(f_in.readlines()):
        if line > 1:
            li = text.split(',')
            try:
                a_x = float(li[3])
                a_y = float(li[4])
                a_z = float(li[5])
            except ValueError:
                continue
            time.append(float(li[0]))
            acc_x.append(a_x)
            acc_y.append(a_y)
            acc_z.append(a_z)

cuttoff = -180000
time = time[cuttoff:]
acc_x = acc_y[cuttoff:]
acc_y = acc_y[cuttoff:]
acc_z = acc_y[cuttoff:]


ystand = median(acc_y[:5000])
yfreefall = median(acc_y[87500:91500])
gain= ystand - yfreefall / 1.0
acc_y = divide(subtract(acc_y, yfreefall), gain)

liftoff_idx = argmax(acc_y[8000:9000])
liftoff = time[8000+liftoff_idx]

time = subtract(time, liftoff)

In [ ]:
# lightly lowpass
from scipy.signal import butter, lfilter, freqz

# Filter requirements.
order = 6
fs = 2500       # sample rate, Hz
cutoff = 190   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_y_filter = lfilter(b, a, acc_y)

In [ ]:
ax = plt.figure(figsize=(16,6))
plt.title(r"IMU Vertical Acceleration")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(time[1000:-1000], acc_y_filter[1000:-1000], alpha=0.7, lw=1.0)
plt.xlim([-4,70])
plt.show()